# Matrix Factorization

在存储　user-item，　计算和存储　user-user　及　item-item 相似度矩阵的时候，最大的问题就是计算和存储的复杂度为　$O(MN)$。这次依然沿用　MovieLens　[数据集](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)。

依然沿用之前的加载数据的过程

In [1]:
import os
import pandas as pd
import numpy as np

# working directory
BASEDIR = os.getcwd()
print(BASEDIR)


dataframe = pd.read_csv(BASEDIR + '/assets/datasets/ml-latest-small/ratings.csv')

userId_dict = {}
movieId_dict = {}

userId_unique = dataframe.userId.unique()
movieId_unique = dataframe.movieId.unique()


idx = 0
for n in range(userId_unique.shape[0]):
    userId_dict[userId_unique[idx]] = idx
    idx += 1

idx = 0
for n in range(movieId_unique.shape[0]):
    movieId_dict[movieId_unique[idx]] = idx
    idx += 1

ratings = np.zeros(shape=(len(userId_dict), len(movieId_dict)))


for row in dataframe.itertuples():
    ratings[userId_dict[row.userId], movieId_dict[row.movieId]] = row.rating

/home/hailingu/Git/MLFM


矩阵分解的话，有几种做法：

- 对方阵, 可以使用　Eigen Decomposition
- 使用　Singular Value Decomposition
- 使用　Gradient Descent

在这个地方，采用　Gradient Descent，使用这个方案的原因就是因为它是之前做法的一种延续。

现在我们需要把　ratings　矩阵分解成两个矩阵，分别是　U 和　M, 其中　U 是一个　$m \times k$　的矩阵，相应的 I 就是一个　$k \times n$　的矩阵， $m, n$ 分别是　user 和　movie 的数量。

对于某一个用户　$i$　对电影　$j$ 的预计打分可以表示为 $\hat{ratings_{ij}} = u_i \cdot m_ｊ^T$，这里是向量的点乘，不是矩阵乘法，所以第二项可以写在前面，也可以写在后面。单个损失函数： $Loss_{ij} = \frac{1}{2}(\hat{ratings_{ij}} - ratings_{ij})^2$。

Gradient 计算及更新：

$\begin{align*}
u_i & = u_i - \eta \cdot (\hat{ratings_{ij}} - ratings_{ij}) \cdot m_j \\
m_j & = m_j - \eta \cdot (\hat{ratings_{ij}} - ratings_{ij}) \cdot u_i
\end{align*}$

如果写成矩阵处理的形式就是：

$\begin{align*}
U & = U - \eta \cdot (U \times M - ratings) \times M^T \\
M & = M - \eta \cdot U^T \times (U \times M - ratings) 
\end{align*}$


In [2]:
k = 150
LEARNING_RATE = 1e-7

EPOCH = 300
U = np.random.uniform(0, 1, size=(len(userId_dict), k))
M = np.random.uniform(0, 1, size=(k, len(movieId_dict)))

PRINT_STEP = EPOCH / 10
for epoch in range(EPOCH):
    hat_ratings = np.dot(U, M)
    loss = hat_ratings - ratings
    U_copy = U.copy()
    M_copy = M.copy()

    U = U - LEARNING_RATE * np.dot(loss, M_copy.T)
    M = M - LEARNING_RATE * np.dot(U_copy.T, loss)

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, (loss * loss).sum()))

recommend_matrix = np.dot(U, M)
print(ratings[1, :])
print(recommend_matrix[1, :])

EPOCH: 0, loss: 8392133697.507488
EPOCH: 30, loss: 928338586.768743
EPOCH: 60, loss: 119758315.245776
EPOCH: 90, loss: 21227786.737734
EPOCH: 120, loss: 9025386.322319
EPOCH: 150, loss: 7486919.731269
EPOCH: 180, loss: 7268526.145244
EPOCH: 210, loss: 7213732.626706
EPOCH: 240, loss: 7179361.876418
EPOCH: 270, loss: 7147686.020820
[0. 0. 0. ... 0. 0. 0.]
[-0.38198986  0.29601525 -0.30486487 ...  0.8878231  -0.57686675
 -0.21223052]


In [3]:
N = 5
top_n_idx = recommend_matrix[1, :].argsort()[::-1][1:N + 1]
print(top_n_idx)

[4557  796 2111 9503  544]
